# 构建一个基于SQL数据的问答系统 | 🦜️🔗 LangChain

[https://python.langchain.com/v0.2/docs/tutorials/sql_qa/](https://python.langchain.com/v0.2/docs/tutorials/sql_qa/)

## 配置

In [1]:
pip install --upgrade --quiet  langchain langchain-community langchain-openai faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install langgraph==0.0.66

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 1.3 MB/s eta 0:00:00


In [ ]:
pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bidict                           0.23.1

在这个教程中我们将使用openai的模型

In [18]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('MOON_API_KAY')
os.environ["OPENAI_API_BASE"] = userdata.get('MOON_API_BASE')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

创建sqlite数据库

In [4]:
import sqlite3
import requests

# Step 1: 下载 Chinook_Sqlite.sql 文件
url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
sql_file_path = "Chinook_Sqlite.sql"

try:
    response = requests.get(url)
    response.raise_for_status()  # Check if the download was successful
    with open(sql_file_path, "wb") as file:
        file.write(response.content)
    print("Downloaded Chinook_Sqlite.sql successfully.")
except requests.exceptions.RequestException as e:
    print(f"Failed to download file: {e}")

# Step 2: 创建连接数据库 Chinook.db
db_path = "Chinook.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Step 3: 读取执行 Chinook_Sqlite.sql 文件
try:
    with open(sql_file_path, "r", encoding="utf-8") as file:
        sql_script = file.read()
    cursor.executescript(sql_script)
    conn.commit()
    print("Executed SQL script and created database successfully.")
except Exception as e:
    print(f"Failed to execute SQL script: {e}")

# Step 4: 查询测试
try:
    cursor.execute("SELECT * FROM Artist LIMIT 10;")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Failed to run test query: {e}")

# 关闭连接
conn.close()


Downloaded Chinook_Sqlite.sql successfully.
Executed SQL script and created database successfully.
(1, 'AC/DC')
(2, 'Accept')
(3, 'Aerosmith')
(4, 'Alanis Morissette')
(5, 'Alice In Chains')
(6, 'Antônio Carlos Jobim')
(7, 'Apocalyptica')
(8, 'Audioslave')
(9, 'BackBeat')
(10, 'Billy Cobham')


我们来看一下使用SQLite连接Chinook数据库，请按照以下安装步骤，在与此笔记本相同的目录中创建Chinook.db：

- 下载保存文件到 `Chinook.sql`
- 执行 `sqlite3 Chinook.db` 命令
- 执行 `.read Chinook.sql` 命令
- 测试查询`SELECT * FROM Artist LIMIT 10;`

现在， `Chinook.db` 已经在我们的目录中，我们可以使用由SQLAlchemy驱动的SQLDatabase类与之交互：

In [5]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT ArtistId FROM Artist WHERE Name = 'Antônio Carlos Jobim'")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


'[(6,)]'

**API 调用:**[SQLDatabase](https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.sql_database.SQLDatabase.html)


现在我们已经有了一个可以查询的 SQL 数据库。现在让我们尝试将它连接到一个 LLM。

## Chains

链（也就是由LangChain的[Runnables](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language)子类组成的组合）适合能够确定执行流程的应用。我们可以先创建一个简单的链，它接受一个问题并执行以下操作：

- 将问题转换成SQL查询语句；
- 执行这个查询；
- 用查询结果来回答最初的问题。

但是链并不支持所有场景。比如，对于任何用户输入哪怕是“hello”，这个系统都会执行一个SQL查询。重要的是，有些问题需要多次查询才能解答。这种场景适用于agent智能体。

### 将问题转换为 SQL 查询语句


在链或agent中，第一步是获取用户输入并将其转换为SQL查询。LangChain提供了一个内置的链来完成这项工作： `create_sql_query_chain`。

In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="moonshot-v1-32k")

In [20]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

'SELECT COUNT(*) \nFROM "Employee";'

**API 调用:**[create_sql_query_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.sql_database.query.create_sql_query_chain.html)

尝试执行sql



In [21]:
db.run(response)

'[(8,)]'


我们可以查看[LangSmith追踪](https://smith.langchain.com/public/c8fa52ea-be46-4829-bde2-52894970b830/r)来更好地理解这个链在做什么。我们也可以直接查看提示词（如下）：

- 明确引用了SQLite。
- 为所有可用的表提供了定义。
- 为每个表提供了三行示例数据。

我们也可以像这样检查完整的提示：

In [22]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result



### 执行 SQL 查询

生成了SQL查询之后，下一步自然是执行它。这是创建SQL链中最危险的一部分。请仔细考虑是否适合在您的数据上运行自动化查询。尽可能地最小化数据库连接权限。可以考虑在执行查询之前为您的链添加人工审批步骤（见下文）。

我们可以使用 `QuerySQLDatabaseTool` 轻松地为链添加执行sql的节点：

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'[(8,)]'

**API 调用:**[QuerySQLDataBaseTool](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.sql_database.tool.QuerySQLDataBaseTool.html)

```
'[(8,)]'
```

### 回答问题

现在我们已经能够自动生成并执行查询，只需要将原始问题和SQL查询结果结合起来，生成最终答案。我们可以通过再次将问题和结果传递给大型语言模型(LLM)来实现：

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
answer_prompt = PromptTemplate.from_template(
"""Given the following user question, corresponding SQL query, and SQL result, answer the user question.
Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
)
| answer_prompt
| llm
| StrOutputParser()
)
chain.invoke({"question": "How many employees are there"})

'There are 8 employees in the database.'

**API 调用：**[StrOutputParser](https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) | [PromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.prompt.PromptTemplate.html) | [RunnablePassthrough](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html)



让我们回顾一下上述LCEL中发生的事情。假设这个链被调用。

- 在第一个`RunnablePassthrough.assign`之后，会返回一个包含两个元素的可运行对象：`{"question": question, "query": write_query.invoke(question)}`，其中`write_query`将生成一个SQL查询语句，以回答这个问题。
- 在第二个`RunnablePassthrough.assign`之后，我们添加了第三个元素`"result"`，它包含`execute_query.invoke(query)`，其中`query`是前一步生成的结果。
- 这三个输入被格式化成提示，并传递给LLM。
- `StrOutputParser()`提取输出消息的字符串内容。

在将大型语言模型（LLMs）、工具、提示和其他链组合在一起时，由于每个都实现了Runnable接口，它们的输入和输出可以向下绑定。

### 后续步骤

对于更复杂的查询生成，我们可能想要创建少量样本提示或添加查询检查步骤。对于更多内容，请查看：

- [提示策略](https://python.langchain.com/v0.2/docs/how_to/sql_prompting/)：高级提示工程技巧。
- [查询检查](https://python.langchain.com/v0.2/docs/how_to/sql_query_checking/)：增加查询验证和错误处理。
- [大型数据库](https://python.langchain.com/v0.2/docs/how_to/sql_large_db/)：处理大型数据库的技术



## Agents

LangChain 有一个 SQL Agent，它提供了一种比链式操作更灵活的与 SQL 数据库交互的方式。使用 SQL Agent 的主要优势包括：

- 它可以根据数据库的模式以及数据库的内容（比如描述一个特定的表）来回答问题。
- 它可以通过运行生成的查询，捕获追踪栈并正确地重新生成，来从错误中恢复。
- 它可以根据需要多次查询数据库以回答用户的问题。
- 它仅从相关表中检索的方式来节省令牌。

为了初始化agent ，我们将使用 `SQLDatabaseToolkit` 来创建一系列工具：

- 创建和执行查询
- 检查查询语法
- 检索表描述
- ...以及更多功能

In [23]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f1cc3bf9630>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f1cc3bf9630>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f1cc3bf9630>),
 QuerySQLCheckerTool(description='Use this tool to double check

**API 调用：**[SQLDatabaseToolkit](https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.sql.toolkit.SQLDatabaseToolkit.html)


### System Prompt 系统提示词

我们还需要为agent创建一个系统提示词。这将包括如何执行的指令。




In [24]:
from langchain_core.messages import SystemMessage
SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""
system_message = SystemMessage(content=SQL_PREFIX)

**API 参考:**[SystemMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.system.SystemMessage.html)

### 智能体 agent 初始化
我们将使用预构建的[LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph) agent来构建我们的代理。

In [25]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

**API 调用：**[HumanMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.human.HumanMessage.html)

In [27]:
for s in agent_executor.stream(
  {"messages": [HumanMessage(content="Which country's customers spent the most?")]}
):
  print(s)
  print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'sql_db_list_tables:0', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 568, 'total_tokens': 571}, 'model_name': 'moonshot-v1-32k', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2dac0ff8-70dc-44f8-95a3-ed7941ee6d1e-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'sql_db_list_tables:0'}], usage_metadata={'input_tokens': 568, 'output_tokens': 3, 'total_tokens': 571})]}}
----
{'tools': {'messages': [ToolMessage(content='Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', name='sql_db_list_tables', tool_call_id='sql_db_list_tables:0')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'sql_db_schema:0', 'function': {'arguments


请注意，agent会执行多个查询，直到它获得所需的信息：

1. 列出可用的表；
2. 检索三个表的架构；
3. 通过连接操作查询多个表。

然后，代理能够使用最终查询的结果来生成原始问题的答案。

agent同样能够处理定性问题：

In [ ]:
for s in agent_executor.stream(
{"messages": [HumanMessage(content="Describe the playlisttrack table")]}
):
  print(s)
  print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0jRwNptf1RHFPxOkUCtpNQuE', 'function': {'arguments': '{"table_names":"playlisttrack"}', 'name': 'sql_db_schema'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 554, 'total_tokens': 571}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d2fc6d74-bf97-4063-856b-7938e35a52c0-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'playlisttrack'}, 'id': 'call_0jRwNptf1RHFPxOkUCtpNQuE'}], usage_metadata={'input_tokens': 554, 'output_tokens': 17, 'total_tokens': 571})]}}
----
{'tools': {'messages': [ToolMessage(content="Error: table_names {'playlisttrack'} not found in database", name='sql_db_schema', tool_call_id='call_0jRwNptf1RHFPxOkUCtpNQuE')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GvCfLVteA8dMa



### 处理查询专有名词（高基数(high-cardinality)列）

具有大量不同值的列。 例如，一个存储员工ID的列，每个员工都有唯一的ID，那么这个列就是高基数列。

为了筛选包含专有名词的列，例如地址、歌曲名称或艺术家，我们首先需要仔细检查拼写，以便正确筛选数据。

我们可以通过创建一个包含数据库中所有不同专有名词的向量存储来实现这一点。然后，每次用户在他们的问题中包含专有名词时，代理就可以查询这个向量存储，以找到那个词的正确拼写。通过这种方式，agent可以确保在构建目标查询之前理解用户所指的实体。

首先，我们需要每个实体的唯一值，为此我们定义了一个函数，该函数将结果解析为元素列表：

In [ ]:
import ast
import re
def query_as_list(db, query):
  res = db.run(query)
  res = [el for sub in ast.literal_eval(res) for el in sub if el]
  res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
  return list(set(res))
artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")
albums[:5]

["Schubert: The Late String Quartets & String Quintet ( CD's)",
 'A TempestadeTempestade Ou O Livro Dos Dias',
 'BBC Sessions [Disc ] [Live]',
 'Appetite for Destruction',
 'My Way: The Best Of Frank Sinatra [Disc ]']


使用这个函数，我们可以创建一个检索工具，代理可以根据自己的判断执行。

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
vector_db = FAISS.from_texts(artists + albums, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)
tools = [retriever_tool] + tools

**API 调用:**[create_retriever_tool](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.create_retriever_tool.html) | [FAISS](https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.faiss.FAISS.html) | [OpenAIEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html)

In [ ]:
print(retriever_tool.invoke("Alice Chains"))

Alice In Chains

Alanis Morissette

Pearl Jam

Pearl Jam

Audioslave


这样，如果agent确定需要根据像“Alice Chains”这样的艺术家来编写过滤器，它首先可以使用检索工具来观察某一列的相关值。

In [ ]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
You have access to the following tables: {table_names}
If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.""".format(
    table_names=db.get_usable_table_names()
)
system_message = SystemMessage(content=system)
agent = create_react_agent(llm, tools, messages_modifier=system_message)

In [ ]:
for s in agent.stream(
{"messages": [HumanMessage(content="How many albums does antônio carlo jobim have?")]}
):
  print(s)
  print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mnBaK1SGfXJlirOBNgn6QgmR', 'function': {'arguments': '{"query":"ant\\\\u00f4nio carlo jobim"}', 'name': 'search_proper_nouns'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 739, 'total_tokens': 766}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f68b292c-de56-4925-9dde-5b6c125297c2-0', tool_calls=[{'name': 'search_proper_nouns', 'args': {'query': 'ant\\u00f4nio carlo jobim'}, 'id': 'call_mnBaK1SGfXJlirOBNgn6QgmR'}], usage_metadata={'input_tokens': 739, 'output_tokens': 27, 'total_tokens': 766})]}}
----
{'tools': {'messages': [ToolMessage(content='Antônio Carlos Jobim\n\nCaetano Veloso\n\nJoão Gilberto\n\nChico Buarque\n\nGonzaguinha', name='search_proper_nouns', tool_call_id='call_mnBaK1SGfXJlirOBNgn6QgmR')]}}
----
{'agent': {'messages': [AIMessage(content='',

In [ ]:
print(tools)

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7d2118a61de0>), InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7d2118a61de0>), ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7d2118a61de0>), QuerySQLCheckerTool(description='Use this tool to double check if


正如我们所见，agent 使用了 `search_proper_nouns` 工具，以便正确查询数据库中这位特定艺术家的信息。